In [1]:
import numpy as np
import soundfile as sf
import librosa
import matplotlib.pyplot as plt
import IPython.display

In [2]:
from utils.evals import sisdr
from utils.others import zero_pad, STFT, iSTFT
from utils.phase_reconstruction import misi, divmisi_ver1

In [3]:
f1, _ = sf.read('../data/f1.wav')
f2, _ = sf.read('../data/f2.wav')

---

In [4]:
winlen = 1024
shift = 256
epsilon = 1e-15

stft = STFT(winlen, shift)
istft = iSTFT(winlen, shift)

In [5]:
f1 = zero_pad(f1, winlen, shift)
f2 = zero_pad(f2, winlen, shift)
fm = f1 + f2

cm = stft(fm);

In [6]:
IPython.display.Audio(f1, rate=16000)
print(np.max(f1-istft(stft(f1))))

1.4901161193847656e-08


---

In [7]:
mask1 = np.abs(stft(f1))**2/(np.abs(stft(f1))**2+np.abs(stft(f2))**2+epsilon)
mask2 = np.abs(stft(f2))**2/(np.abs(stft(f1))**2+np.abs(stft(f2))**2+epsilon)

In [8]:
masked1 = mask1*cm
masked2 = mask2*cm
f1est_noisy = istft(masked1)
f2est_noisy = istft(masked2)

In [9]:
print((sisdr(f1, f1est_noisy)+sisdr(f2, f2est_noisy))/2)
IPython.display.Audio(f1est_noisy, rate=16000)

9.708891261824569


---

In [10]:
f1est_misi, f2est_misi = misi(masked1, masked2, fm, stft, istft)

In [11]:
print((sisdr(f1, f1est_misi)+sisdr(f2, f2est_misi))/2)
IPython.display.Audio(f1est_misi, rate=16000)

9.712382204393258


In [12]:
f1est_dmisi, f2est_dmisi =divmisi_ver1(masked1, masked2, fm, stft, istft)

In [13]:
print((sisdr(f1, f1est_dmisi)+sisdr(f2, f2est_dmisi))/2)
IPython.display.Audio(f1est_dmisi, rate=16000)

13.378877072729173
